# デーモンスレッド使用時のスクリプト終了時の例外対応

## デーモンスレッド使用時の問題点
### デーモンスレッドとは

Pythonのデーモンスレッドは非デーモンスレッドが全て終了すると強制的に終了される。この性質は、ソケットなどによるプロセス間通信をバックグラウンドで実行するようなコマンドラインアプリケーションで用いられる。Pythonの引数に指定したスクリプトファイルの終端に達したり、あるいは例外によりメインスレッドが終了した場合に、バックグラウンドで走行しているスレッドも終了しプロンプトに戻る。もしデーモンを用いてなければ、そのスレッドが終了するまでプロセスは延々と生きてしまう。ソケットで受信待機するようなスレッドであれば、リモートからコネクションが切られでもしない限りプロンプトには戻らない。

### 発生する問題

デーモンスレッドを用いたコマンドラインアプリケーションで、以下のような例外が発生することがある。同じスクリプト、同じデータでも発生することもあれば発生しないこともあり、事前に予測できない。

```
Exception in thread XXXX (most likely raised during interpreter shutdown):
Traceback (most recent call last):
  ...
<type 'exceptions.AttributeError'>: 'NoneType' object has no attribute 'xxxx'
```

例外のメッセージは、メインスレッド(及び全ての非デーモンスレッド)が終了してPythonインタープリタのシャットダウン(終了、クリーンアップ)中に発生した可能性を示唆している。くだけて言うと「たぶんにクリーンアップ処理が起因で発生したものでアプリの問題ではないよ」とでもなろうか。

ちなみに、実際に上のような例外が起きたときのPythonの動作は ```traceback.print_exc()``` で、```import```済みのモジュール ```traceback``` の ```print_exc``` 関数を呼び出そうとして発生した。「```traceback``` は ```None``` オブジェクトを指しているので ```print_exc``` という属性は持っていない」という一見不可思議な例外である。

この例での例外は、```AttributeError``` であるが ```TypeError``` が発生するこもあり、例外の種別もこれと予測できるものではない。

### 対処方法の模索

例外が発生することを防げないようなので、例外をキャッチして無視する他ない。無条件に無視すると、インタープリタの終了時ではなく本当にアプリケーション側に問題があった場合に検出できなくなる。そこで、```atexit``` で終了処理に入ったことを記憶しておき、例外補足時にそれをもとに無視するかどうかを決めるようにしてみた。おおむね、以下のようなコードになる。

```python
_IN_SHUTDOWN = False

def _mark_shutdown():
    global _IN_SHUTDOWN
    _IN_SHUTDOWN = True
atexit.register(_mark_shutdown)

def _background_procedure_in_daemon_thread():
    try:
         # ... action ...
    except:
        if not _IN_SHUTDOWN:
            raise
```

しかし、この方法では ```_mark_shutdown``` が呼ばれた後にもかかわらず `if not _IN_SHUTDOWN: raise` が実行されてしまった。

> 実は \_IN_SHUTDOWN ではなく \_IN_RUNNING のように真偽を逆転させるとうまくいく。理由はこのあとで判明する。

## インタープリタ終了時に何が起きているのか

### メッセージ表示箇所を突き止める

インターネットで検索してみると、この問題に対しては「デーモンスレッドを使う以上仕方ない」「デーモンスレッドを使うべきではない」という回答しか得られない。そこで、Pythonのソースコードからこの現象が起きる条件を探った。

これらの例外で必ず表示される文字列 ```most likely raised during interpreter shutdown``` を検索することであたりをつけようとしたが、予想に反し、部分文字列も含めこの文字列に該当する箇所は ```.[ch]``` には存在せず、Pythonの標準モジュール `threading.py` でこれが見つかった。次のコードがその該当個所である。

```python
            try:
                self.run()
            except SystemExit:
                if __debug__:
                    self._note("%s.__bootstrap(): raised SystemExit", self)
            except:
                if __debug__:
                    self._note("%s.__bootstrap(): unhandled exception", self)
                # If sys.stderr is no more (most likely from interpreter
                # shutdown) use self.__stderr.  Otherwise still use sys (as in
                # _sys) in case sys.stderr was redefined since the creation of
                # self.
                if _sys and _sys.stderr is not None:
                    print>>_sys.stderr, ("Exception in thread %s:\n%s" %
                                         (self.name, _format_exc()))
                elif self.__stderr is not None:
                    # Do the best job possible w/o a huge amt. of code to
                    # approximate a traceback (code ideas from
                    # Lib/traceback.py)
                    exc_type, exc_value, exc_tb = self.__exc_info()
                    try:
                        print>>self.__stderr, (
                            "Exception in thread " + self.name +
                            " (most likely raised during interpreter shutdown):")
                        print>>self.__stderr, (
                            "Traceback (most recent call last):")
```
このコードは `Thread.__bootstrap_inner` の一部で、`self.run()` でスレッドオブジェクト生成時の `target` 引数に指定した関数を実行し、スレッド処理で発生した例外を補足している。インタープリタ終了に伴って発生した例外は最悪ここで補足され、そして、わざわざ `most likely raised during interpreter shutdown` と表示してくれているという訳だ。

この表示を行なう条件のみおおざっぱに取り出してみると以下のようになる。
```python
import sys as _sys

class Thread(_Verbose):
    __exc_clear = _sys.exc_clear

    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        # ...
        self.__stderr = _sys.stderr

    def __bootstrap_inner(self):
        # ...
            except:
                 if _sys and _sys.stderr is not None:
                    print>>_sys.stderr, ("Exception in thread %s:\n%s" %
                                         (self.name, _format_exc()))
                elif self.__stderr is not None:
                    # Do the best job possible w/o a huge amt. of code to
                    # approximate a traceback (code ideas from
                    # Lib/traceback.py)
                    exc_type, exc_value, exc_tb = self.__exc_info()
                    try:
                        print>>self.__stderr, (
                            "Exception in thread " + self.name +
                            " (most likely raised during interpreter shutdown):")

```
このコードの中程にある `if _sys and _sys.stderr` がインタープリタ終了中かどうかの目安として使われていそうなことがわかる。ただ、そもそも `_sys` はこの `threading.py` の先頭の方で `import sys as _sys` として導入されたグローバルな名前である。にもかかわらず何故`_sys` の真偽を判定しているのか。

### 言語仕様には

実は後で気付いたことであるが、言語仕様の `3.4.1. 基本的なカスタマイズ` の `__del__` メソッドの説明に以下のような **警告** の記載があった。

>警告
>
>`__del__()` メソッドの呼び出しが起きるのは不安定な状況下なので、 `__del__()` の実行中に発生した例外は無視され、代わりに `sys.stderr` に警告が出力されます。また、 (例えばプログラムの実行終了による) モジュールの削除に伴って `__del__()` が呼び出される際には、 `__del__()` メソッドが参照している他のグローバル変数はすでに削除されていたり、削除中(例えば、`import`機構のシャットダウン中)かもしれません。この理由から、 `__del__()` メソッドでは外部の不変関係を維持する上で絶対最低限必要なことだけをすべきです。バージョン 1.5 からは、**単一のアンダースコアで始まるようなグローバル変数は、他のグローバル変数が削除される前にモジュールから削除されるように Python 側で保証しています**; これらのアンダースコア付きグローバル変数は、 `__del__()` が呼び出された際に、`import` されたモジュールがまだ残っているか確認する上で役に立ちます。 

この仕様では **変数が削除** と書かれているが、`threading` のコードは「変数は残っているが、変数の指すオブジェクトが偽判定となるオブジェクトに変更されている」事に依存していて、**削除**の意味の曖昧さがある。

### ソースコードでは

`threading.py` のコードだけでは釈然とせず、また可能な限り良い対処方法を見いだすために、インタプリタ終了処理のソースコードも調査した。スクリプトの終端に到達あるいは例外が発生したあと `Py_Finalize` が呼ばれる。この関数がインタプリタ終了処理の入り口となっている。

#### Py\_Finalize
この関数は終了処理の全体を制御している。おおむね以下のような処理が行われる。
- `wait_for_thread_shutdown()` で全ての非デーモンスレッドの終了を待つ。
- `call_sys_exitfunc()` で `sys.exitfunc` が存在すれば呼び出す。`import atexit` すると `sys.exitfunc` に `atexit._run_exitfuncs` が設定される。この関数は `atexit.register` で登録した関数を呼び出す。
- `PyImport_Cleanup()` でモジュールのクリーンアップを行う。。
- python 内部に組み込まれている基本型のクリーンアップを行う。
- `call_ll_exifuncs()` で `Py_AtExit` とう登録した関数が呼ばれる。

Pythonのデバッグ情報の出力などを見たところ、問題となっている例外は `PyImport_Cleanup()` 呼び出しの最中か、あるいはその後の基本型のクリーンアップ中に発生している。

#### PyImport\_Cleanup
この関数では非明示的なものも含めて `import` されたモジュールオブジェクトのクリーンアップが行われる。
- `sys`モジュールの辞書に対して、
 + ある種の名前(`path`, `argv`, ...)について、その名前に対して `None` を割り当てる。`.c` の実装において、名前に `None` を割り当てる処理というのは、割り当て前に名前が指していたオブジェクトの参照カウントを減らしたあと、`Py_None` を割り当てるという意味である。
 + `sys.std*` (`*` は `in`, `out`, `err`) に対して、`sys.__std*__` を割り当てる。
- `__main__` モジュールについて `_PyModule_Clear()` を呼び出し、`__main__` に `None` を割り当てる。
- `__builtin__` と `sys` を除いて、参照カウントが1のモジュールが見つかる間、見つかったモジュールに対して `_PyModule_Clear()` を呼び出す。
- `__builtin__` と `sys` を除いた残りの全てのモジュールについて `_PyModule_Clear()` を呼び出す。
- `sys`, `__builtin__` の順で `_PyModule_Clear()` を呼び出す。

#### \_PyModule\_Clear
この関数はモジュールオブジェクトから他のオブジェクトへの参照を切る処理を、以下の順で行う。
- モジュールの辞書にある名前のうち、単一のアンダースコア `_` で始まる名前に対して `None` を割り当てる。
- モジュールの辞書にある名前のうち、`__builtins__` 以外の名前に `None` を割り当てる。

言語仕様にあった記述はどうもこの部分のように思われる。

## 対処方法の考察

対処方法として例外を補足した時に、インタープリタが終了処理中であれば例外を無視するという事はそのままとする。問題となるのは、その判断方法である。

### グローバル変数が None になる性質を利用する [不十分]
`_PyModule_Clear` の動作を知れば、以下のようなコードが考えられる。
```python
_IN_RUNNING = True

def _background_procedure_in_daemon_thread():
    try:
         # ... action ...
    except:
        if _IN_RUNNING:
            raise
```
このコードの問題点は、インタープリタの終了処理に入ったが、このモジュールに対する `_PyModule_Clear` が呼ばれる前に例外が発生する場合がある点である。

### グローバル変数が None になる性質＋atexit [まあまあ]
前のコードに `atexit` による明示的な変数クリアを追加する方法である。
```python
_IN_RUNNING = True

def _mark_shutdown():
    global _IN_RUNNING
    _IN_RUNNING = None
atexit.register(_mark_shutdown)

def _background_procedure_in_daemon_thread():
    try:
         # ... action ...
    except:
        if _IN_RUNNING:
            raise
```
`atexit.register` で登録された関数はモジュールのクリーンアップ処理よりも前に呼び出されるため以前の方法による問題は回避される。

## 改善

インタープリタ終了時のグローバル変数の性質を用いる方法は手軽であるが、グローバル変数がモジュールの外側からでも容易に変更・削除されやすい性質を持つことを考えると避けたい。そこで、変更されにくく削除もできない変数であるローカル変数を経由することを考える。また、スレッドのターゲット関数を定義する各モジュールに `_mark_shutdown` などのコードを重複して記述しないように独立したモジュール内(ここでは、`threadutil.py` としておく)に実装することを考える。

### threadutil.py
以下のコードを準備する。
```python
def quiet_finalize():
    import atexit
    import functools

    class Monitor(object):
        pass
    monitor = Monitor()
    monitor.running = True

    @atexit.register
    def go_shutdown():
        monitor.running = False

    def is_running(*args):
        # args is never used but this is required for simplified coding
        # in case of assigning threadutil.is_running to a class attribute.
        return monitor.running

    def decorator(target):
        @functools.wraps(target)
        def _target(*argv, **kws):
            try:
                return target(*argv, **kws)
            except:
                if monitor.running:
                    raise
        return _target

    return decorator, is_running

quiet_finalize, is_running = quiet_finalize()

```
`def` で定義されている `quiet_finalize` はインタープリタ終了判定に関係するクロージャ変数やクロージャ関数を生成するためのものである。この `threadutil.py` の `import` 時に `quiet_finalize` は実行される。この関数の中で、インタープリタが走行中(終了処理に入っていないという意味)かどうかを保持する `Monitor` オブジェクトを生成し、ローカル変数にその参照を残す。このローカル変数は `go_shutdown`, `decorator`, `is_running` の3つのネストした関数から参照される。走行状態の `running` を直接 `quiet_finalize` のローカル変数にせず `Monitor` オブジェクトを介しているのは、Python(ここでは2系)の言語仕様上、関数内での `global` 指定のない変数への代入は、その関数内のローカル変数として解釈されるためである。もし、`go_shutdown` で `running` への代入を行うと、`quiet_finalize` の `running` が `go_shutdown` で共有されなくなる。

`quiet_finalize()` は内部の関数である `decorator` と `is_running` を戻し、これをモジュールトップレベルの `quiet_finalize`, `is_running` と紐付ける。ここで `quiet_finalize` を上書きしているのは、`def` で定義したもともとの `quiet_finalize` 関数が二度呼び出されることを避けるためである(関数オブジェクトやコードオブジェクトをインスペクションすれば、もとの `quiet_finalize` のコードを呼び出すことは可能であるが、ここまでするのはもはや病的である)。

### quiet\_finalize の使い方
新しい `quiet_finalize` は `threading.Thread` の `target` に指定する関数に対するデコレータとして使用でき、デコレートするだけで most likely raised during interpreter shutdown のメッセージを回避できる。
```python
@quiet_finalize
def daemon_thread():
  # ...
  pass
  
t = threading.Thread(target=daemon_thread)
t.daemon = True
t.start()

```

### is\_running の使い方
`quiet_finalize` を用いることで、デコレート対象の関数から上がる例外を正しく対処することができるが、`target` に指定する関数から深くネストした箇所で例外を補足し処理するケースもありえる。`is_running` はそのような関数で用いることを想定している。ただし注意すべき点がある。例えば、次のように記述したとする。
```python
from threadutil import is_running as _is_running

def action(...):
    try:
        # do something ...
    except:
        if _is_running():
            # error handling
```
これは、インタープリタ終了時のモジュールのグローバル変数の変更により `_is_running` が `None` になっている場合があるため良くない。

`action` が何らかのクラスのメソッドであれば次のようにするのが良い。
```python
class Something(object):
    __is_running = _is_running
    
    def action(self, ...):
        try:
            # do something ...
        except:
            if self.__is_running():
                # error handling
```
なお、`_is_running` へのアクセスには `self` を経由しなければならない。`Something.__is_running` とすると、モジュールグローバルな `Something` を参照することになり、`Something` が `None` に変更されている可能性があり問題の回避とならない。

`action` がモジュールトップレベルの関数として定義されている場合は次のようにクロージャ化するのが良いだろう。
```python
def action(is_running):
    def _action(self, ...):
        try:
            # do something ...
        except:
            if is_running():
                # error handling
    return _action
action = action(_is_running)
```